In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)
import script as sc

In [3]:
import pandas as pd
import json
import sklearn.svm as svm
from sklearn.linear_model import Lasso, LassoCV
from sklearn.feature_selection import SelectFromModel
import numpy as np
from sklearn.neural_network import MLPClassifier

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [4]:
## Change the name to get your path
user = 'Andrew'

## Build test and train Datasets

In [5]:
data = pd.DataFrame(sc.get_complete(user, 'complete_wide.csv'))

In [6]:
data.head()

,Unnamed: 0,Unnamed: 0.1,X1,X2,X3,X4,X5,X6,X7,X8,...,X238,X239,X240,X241,Unnamed: 0.2,observation,date,activity,coding,time
0,0,1.0,2017-10-06 16:44:00,-0.191,-0.711,-0.660,-0.172,-0.715,-0.652,-0.207,...,-0.648,-0.281,-0.730,-0.648,1.0,H_AM01DO1_M,13-01-2018,active,non-sedentary,0.0
1,1,2.0,2017-10-06 16:44:01,-0.254,-0.715,-0.645,-0.266,-0.727,-0.637,-0.289,...,-0.711,0.035,-0.707,-0.715,2.0,H_AM01DO1_M,13-01-2018,active,non-sedentary,1.0
2,2,3.0,2017-10-06 16:44:02,0.063,-0.715,-0.719,0.129,-0.695,-0.719,0.164,...,-0.301,-0.828,0.180,-0.316,3.0,H_AM01DO1_M,13-01-2018,active,non-sedentary,2.0
3,3,4.0,2017-10-06 16:44:03,-0.773,0.180,-0.305,-0.754,0.219,-0.285,-0.777,...,-0.160,-1.039,0.332,-0.156,4.0,H_AM01DO1_M,13-01-2018,active,non-sedentary,3.0
4,4,5.0,2017-10-06 16:44:04,-1.031,0.332,-0.160,-1.016,0.344,-0.164,-1.016,...,0.105,-0.570,0.605,0.133,5.0,H_AM01DO1_M,13-01-2018,active,non-sedentary,4.0


## Reshape and Sample Data

In [7]:
X = data.iloc[:, [i for i in range(3, 243)]]
criterion = data.iloc[:, [246]]

In [16]:
## creating variables for x, y, z variables
## If using the file from MAKE_WIDE, this step shouldn't be necessary
x = []
y = []
z = []
for i in range(2,242,3):
    
    x.append('X' + str(i))
    y.append('X' + str(i+1))
    z.append('X' + str(i+2))

three_dim_data = np.array([np.array(X[x]),np.array(X[y]),np.array(X[z])])

In [9]:
transpose_data = three_dim_data.transpose()
transpose_data = transpose_data.reshape(100405, 80, 3)

### Get normalized vector magnitudes for each 1/80 second

In [12]:
def curry_norm(x):
    return pd.Series(np.apply_along_axis(np.linalg.norm, 1, x))

In [13]:
vm_data = pd.DataFrame([curry_norm(transpose_data[0])])

In [12]:
for row in transpose_data[1:]:
    vm_data = vm_data.append(curry_norm(row), ignore_index=True)

In [14]:
vm_data.to_csv("vector_magnitudes.csv")

In [23]:
# Use this if you've already saved precomputed vector magnitudes, as above
vm_data = pd.read_csv('vector_magnitudes.csv')

### Append criterion info to vector magnitude data

In [6]:
vm_data = vm_data.iloc[:,1:]

In [7]:
vm_data_crit = pd.concat([vm_data, criterion], axis = 1)

## KNN With DTW

In [17]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.neighbors import KNeighborsClassifier
from fastdtw import *

In [18]:
def get_fastdtw(x, y):
    return fastdtw(x, y)[0]

In [27]:
vm_data_crit['activity'] = criterion

In [11]:
vm_data_crit = vm_data_crit.sample(frac=1)

vm_criterion = vm_data_crit['activity'].copy(deep=True)

vm_criterion[vm_data_crit['activity'] != 'active'] = 0
vm_criterion[vm_data_crit['activity'] == 'active'] = 1

In [14]:
vm_criterion.unique()

array([1, 0], dtype=object)

In [12]:
n = int(0.75 * len(data))

trainX = vm_data_crit.iloc[:n, :80]
testX = vm_data_crit.iloc[n:, :80]

trainY = vm_criterion[:n].values.reshape(n,).astype('float')
testY = vm_criterion[n:].values.reshape((len(data) -n),).astype('float')

In [13]:
dtw = KNeighborsClassifier(n_neighbors = 10, metric=get_fastdtw)

In [ ]:
dtw.fit(trainX, trainY)

KNeighborsClassifier(algorithm='auto', leaf_size=30,
           metric=<function get_fastdtw at 0x000001D579A44F28>,
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [ ]:
dtw_results = dtw.predict(testX)
dtw_results.to_csv("dtw_results.csv")

In [ ]:
dtw_results